# This code calculates the angles needed to correct the BRDF and topo effects from the drone data orthomosaic and the metadata



In [6]:
#rasterio: For working with georeferenced TIFF images.
#pvlib: For solar position calculations.
#pyproj: For coordinate transformations.

%pip install numpy rasterio pvlib pyproj


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.2/29.2 MB 38.0 MB/s eta 0:00:00:00:010:01
Note: you may need to restart the kernel to use updated packages.


In [7]:
import rasterio
from rasterio.transform import xy
import pandas as pd
from pvlib.solarposition import get_solarposition
from pyproj import Geod
import numpy as np
from rasterio.transform import from_origin

In [8]:
image_path = '/home/jovyan/data-store/cross-sensor-cal/data_set2/aligned_orthomosaic.tif'

In [9]:
#Extract pixel coordinates (latitude and longitude) from the georeferenced TIFF.

# Open the georeferenced TIFF
with rasterio.open(image_path) as src:
    # Get the dimensions of the image
    width, height = src.width, src.height
    
    # Get the geotransform and CRS
    transform = src.transform
    crs = src.crs
    
    # Extract pixel coordinates
    coords = [
        xy(transform, row, col, offset='center')
        for row in range(height) for col in range(width)
    ]

    # Separate longitude and latitude
    longitudes, latitudes = zip(*coords)

print(f"Number of pixels: {len(longitudes)}")


Number of pixels: 684096


In [12]:
# Use the pvlib.solarposition.get_solarposition function for solar angles based on location and time.

# Define the image capture date and time
image_datetime = "2023-08-14 21:28:00"  # Example datetime in UTC (locat time of data collection is 14:28)

# Convert to pandas datetime for processing
times = pd.to_datetime([image_datetime])

# Create a DataFrame to store results
solar_angles = []

# Calculate solar angles for each pixel
for lat, lon in zip(latitudes, longitudes):
    # Get solar position
    solar_pos = get_solarposition(times, latitude=lat, longitude=lon)
    #print(solar_pos)
    solar_angles.append({
        "latitude": lat,
        "longitude": lon,
        "solar_zenith": solar_pos["zenith"].values[0],  # Solar Zenith Angle
        "solar_azimuth": solar_pos["azimuth"].values[0],  # Solar Azimuth Angle
    })

# Convert results to a DataFrame
import pandas as pd
solar_angles_df = pd.DataFrame(solar_angles)

print(solar_angles_df.head())


      latitude   longitude  solar_zenith  solar_azimuth
0  4431928.625  453838.875     49.372730     334.488193
1  4431928.625  453839.125     49.465158     334.193230
2  4431928.625  453839.375     49.558574     333.899215
3  4431928.625  453839.625     49.652972     333.606152
4  4431928.625  453839.875     49.748346     333.314046


In [ ]:
#To save the calculated angles as raster layers:

# Define metadata for the new raster
meta = src.meta.copy()
meta.update(dtype='float32', count=2)  # One band for solar zenith, one for azimuth

# Create raster with solar zenith and azimuth
with rasterio.open("solar_angles.tif", "w", **meta) as dst:
    dst.write(solar_angles_df["solar_zenith"].values.reshape(height, width).astype('float32'), 1)
    dst.write(solar_angles_df["solar_azimuth"].values.reshape(height, width).astype('float32'), 2)


### Sensor zenith and azimuth angles depend on the satellite/sensor metadata. This typically includes:

#### Satellite viewing geometry: Look angles, offsets, and swath details. Sensor position and orientation: Often available in metadata files (e.g., for Sentinel-2 or Landsat). Example: Simplified Sensor Zenith and Azimuth Calculation If the sensor's position is given in Earth-Centered Earth-Fixed (ECEF) coordinates and the image's ground coordinates are in latitude/longitude, you can calculate sensor angles using vector geometry.

### this is if not nadir data collection

In [ ]:

# Example sensor position in ECEF (example values, adjust for your sensor)
sensor_position_ecef = np.array([6871.0, -23.0, 1.0])  # x, y, z in km

# Convert latitude and longitude to ECEF
geod = Geod(ellps="WGS84")
sensor_angles = []

for lat, lon in zip(latitudes, longitudes):
    # Convert lat/lon to ECEF
    lon_rad, lat_rad = np.radians(lon), np.radians(lat)
    x = np.cos(lat_rad) * np.cos(lon_rad)
    y = np.cos(lat_rad) * np.sin(lon_rad)
    z = np.sin(lat_rad)
    pixel_position_ecef = np.array([x, y, z])

    # Compute vector between sensor and pixel
    vector = pixel_position_ecef - sensor_position_ecef
    distance = np.linalg.norm(vector)
    
    # Sensor Zenith Angle
    sensor_zenith = np.arccos(vector[2] / distance)  # Angle with respect to Z-axis

    # Sensor Azimuth Angle
    sensor_azimuth = np.arctan2(vector[1], vector[0])  # Angle in XY plane
    
    sensor_angles.append({
        "sensor_zenith": np.degrees(sensor_zenith),
        "sensor_azimuth": np.degrees(sensor_azimuth)
    })

# Combine results with solar angles
for i, sa in enumerate(sensor_angles):
    solar_angles_df.loc[i, "sensor_zenith"] = sa["sensor_zenith"]
    solar_angles_df.loc[i, "sensor_azimuth"] = sa["sensor_azimuth"]

print(solar_angles_df.head())


### this is if nadir data collection

In [13]:
# Example: latitudes and longitudes of pixels
latitudes = [34.05, 34.10, 34.15]  # Example latitudes (degrees)
longitudes = [-118.25, -118.20, -118.15]  # Example longitudes (degrees)

# Placeholder DataFrame for solar angles
solar_angles_df = pd.DataFrame({
    "latitude": latitudes,
    "longitude": longitudes
})

# Assume Nadir Sensor Position
for i in range(len(latitudes)):
    # Sensor zenith angle is 0° in nadir position
    solar_angles_df.loc[i, "sensor_zenith"] = 0.0

    # Sensor azimuth angle can be set to 0° or ignored
    solar_angles_df.loc[i, "sensor_azimuth"] = 0.0

print(solar_angles_df)


   latitude  longitude  sensor_zenith  sensor_azimuth
0     34.05    -118.25            0.0             0.0
1     34.10    -118.20            0.0             0.0
2     34.15    -118.15            0.0             0.0


In [14]:
# Example sensor offset angle from nadir (degrees)
theta_off = 5.0  # Small off-nadir angle

for i, (lat, lon) in enumerate(zip(latitudes, longitudes)):
    # Adjust sensor zenith by the off-nadir angle
    solar_angles_df.loc[i, "sensor_zenith"] = theta_off
    
    # Assume azimuth as 0° for simplicity in this case
    solar_angles_df.loc[i, "sensor_azimuth"] = 0.0

print(solar_angles_df)


   latitude  longitude  sensor_zenith  sensor_azimuth
0     34.05    -118.25            5.0             0.0
1     34.10    -118.20            5.0             0.0
2     34.15    -118.15            5.0             0.0
